## Environment

In [28]:
import numpy as np
import time
import math
import random
import matplotlib.pyplot as plt 
from scipy.optimize import minimize, Bounds, linprog
# Environment parameters
K = 1000  # Number of content items
u_min = 0.2  # Threshold for content relevance
C = int(0.2 * K)  # Number of cached items
#C = 2
# User model parameters
N = 10 # Number of recommended items
q = 0.2  # Probability of ending the viewing session
alpha = 0.8  # Probability of selecting a recommended item
#tradeoff_factor=0.4
# Generate random relevance values
U = np.random.rand(K, K)
np.fill_diagonal(U, 0)  # Set diagonal elements to 0
# U = np.array([[0., 0.09709217, 0.95697935, 0.76421269, 0.79379138],
#               [0.85679266, 0., 0.73115609, 0.97025111, 0.00706508],
#               [0.38327773, 0.27582305, 0., 0.40938946, 0.70918518],
#               [0.27415892, 0.89691232, 0.47103534, 0., 0.97776446],
#               [0.06699551, 0.96500574, 0.00547615, 0.74654658, 0.]])
# U = np.array([[0.0, 0.8, 0.3, 0.6],
#               [0.8, 0.0, 0.7, 0.2],
#               [0.3, 0.1, 0.0, 0.2],
#               [0.6, 0.4, 0.2, 0.0]])
print(U)
#vector to denote the cost of each state. 1 for non-cached, 0 for cached
Cost = [1]*(K-C) +[0]*C   
#Cost = [1,0,1,0]
random.shuffle(Cost)
print(Cost)


def normalize_matrix_R(R):
    row_sums = R.sum(axis=1)

    # New matrix is old matrix divided by row sums
    # Use np.newaxis to match the dimensions for broadcasting
    R = N*R / row_sums[:, np.newaxis]
    return R

# R = normalize_matrix_R(U)
# print(R.sum(axis=1))

[[0.         0.47054753 0.44459091 ... 0.53608153 0.74222978 0.06254457]
 [0.17419118 0.         0.98244361 ... 0.3590356  0.30560787 0.33516673]
 [0.12224103 0.28761012 0.         ... 0.67181377 0.68995107 0.84607762]
 ...
 [0.86080136 0.56732811 0.47839141 ... 0.         0.60527849 0.66032519]
 [0.61543139 0.62544033 0.0268665  ... 0.25022509 0.         0.52538823]
 [0.47357053 0.99779991 0.127991   ... 0.02558122 0.98169621 0.        ]]
[0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

## SLateQ


In [29]:




def all_recommendations_are_relevant(recommendations,s):
    """
    function to check whether everey recommended state in the racommendation batch is 
    relevant to s

    arguments:
    recommendations (tuple of ints): recommendation batch for state s
    s (int): current state

    returns:
    True: if all recommendation are relevant to s
    False: otherwise
    """
    for u in recommendations:
        if U[s][int(u)]<u_min:
            return False
    return True

def func(x,Q):
        return -np.dot(x , Q )/N

def maximize_model(Q,s):
    """
    Function to maximize sum(x_i*1/N*Q(s,i))
    arguments:
    s (int): current state
    Q vector of floats: Q(s,:) values

    returns:
    max value of functio
    argmax of fun
    """

    new_P = np.zeros((K, K), dtype=np.float64) #create a Q value array
    # # Print the result
    # print(f"Minimized function value: {result.fun}")
    # print(f"Argmin: {result.x}")

    
    C=-1/N*Q
    # Equality constraint for summation: sum(x) = N
    A_eq = [[1] * K]
    b_eq = [N]

    # Bounds for each variable: 0 <= x_j <= 1 and x_i = 0
    bounds = [(0, 1) if j != s else (0, 0) for j in range(K)]

    # Now use linprog
    result = linprog(C, bounds=bounds, A_eq=A_eq, b_eq=b_eq, method='highs')
    new_P = result.x
    max = -result.fun
    return new_P,max



def SlateQ(gamma, epsilon,learning_rate):
    """
    Function to perform SlateQ algorithm.

    arguments:
    gamma (float): discounting factor
    epsilon (float): epslilon greedy probability
    learning_rate (float): learning rate

    returns:
    Q (matrix K x K):  martix of state action value function calculated using bellman equation
    """
    Q = np.zeros((K,K)) 
    P = normalize_matrix_R(U) 
    #prev_Q = np.zeros((K,K))
    t = 0
    while True:
        s = np.random.randint(K) #random initial state
        while True:
            if np.random.uniform() < epsilon:  # Explore if e(t) times
                
                numbers = list(range(K))
                numbers.remove(s)

                # Sample N distinct integers from the list
                a = tuple(random.sample(numbers, N))
                #i = random.sample(numbers,1)[0] #choose random action
                    
            else:  # Exploit 1-e(t) times
                a= tuple(np.argpartition(-np.array(P[s]), N)[:N])
                #i = np.argmax(Q[s]) #choose greedily the action with highest Q value
            # a = action_table[s][a_idx]  

            if (all_recommendations_are_relevant(a,s)):
            
                if np.random.uniform() < alpha:  # If all recommended items are relevant
                    i = int(np.random.choice(a))  # Pick a random item from relevant recommended items
                else:  # If at least one recommended item is not relevant
                    i = np.random.randint(K)  # Pick a random item
            else:
                i = np.random.randint(K)  # Pick a random item

            if U[s][i]>u_min:
        
                reward = (1 - 2*Cost[i])
            else:
                
                reward = -1
            #reward = (1 - 2*Cost[i])
            s_prime = i
            if np.random.uniform() < q: #if user opt to terminate session
                target = reward
                Q[s][i] = Q[s][i] + learning_rate * ( target - Q[s][i] )
                break
            else:
                P[s_prime],maxval = maximize_model(Q[s_prime],s_prime)
                target = reward + gamma*maxval
            Q[s][i] = Q[s][i] + learning_rate * ( target - Q[s][i] )
            
            s = s_prime
        t+=1
        epsilon = (t+1)**(-1/3)*(K*math.log(t+1))**(1/3)
        #epsilon = 0.1
        #learning_rate = learning_rate*(1/t)**(1/2)
        
        #if (np.max(np.abs(prev_Q - Q)) < delta and t>1000*K) or 
        if t > 100*K: #check if the new V estimate is close enough to the previous one;
            break # if yes, finish loop
        #prev_Q = Q.copy()
    # print(Q)
    # print("===================================================================================")
    # print(P)
    return P



# pi_Q_learning  =  np.zeros((K, N), dtype=np.int16)
# print(U)
# print(Cost)
# print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
Q = SlateQ(1-q,1,0.01)

pi_SlateQ = np.argpartition(Q, -N, axis=1)[:, -N:]
# for s in range(K):
#     #Q[s][s] = float('-inf')
#     action = np.argmax(Q[s])
#     pi_Q_learning[s] = action_table[s][action]

print(pi_SlateQ)

[[429 488 123 ... 361  17 498]
 [118 680 109 ... 732 209 343]
 [685 742 133 ... 471 595 110]
 ...
 [281 710 598 ... 490 851  58]
 [488  36 118 ... 845 648 727]
 [519 209 129 ... 836 647 831]]


## Simulation

In [30]:
def simulate_session(policy, max_steps=1000):
    """
    Simulate a viewing session following a given policy

    arguments:
    policy to be simulated

    returns:
    total cost of the session
    
    """
    s = np.random.randint(K)  # random initial
    cost_total = Cost[s]  
    for _ in range(max_steps):
        if np.random.uniform() < q:  # The user decides to quit
            break

        if (all_recommendations_are_relevant(policy[s],s)):
            
            if np.random.uniform() < alpha:  # If all recommended items are relevant
                s_prime = int(np.random.choice(policy[s]))  # Pick a random item from relevant recommended items
            else:  # If at least one recommended item is not relevant
                s_prime = np.random.randint(K)  # Pick a random item
        else:
            s_prime = np.random.randint(K)  # Pick a random item
        
        s=s_prime
        cost_total += Cost[s]  # Add the cost of the picked item
    return cost_total

def simulation(policy):
    """
    function to run multiple sessions
    """
    total_cost = 0
    num_of_episodes=50000
    for _ in range(num_of_episodes):
        total_cost  += simulate_session(policy)

    print(total_cost/num_of_episodes)

# print(U)
# print(Cost)
# #print(P_opt1)
# print(pi_SlateQ)

print("average cost for SlateQ:")
simulation(pi_SlateQ)

average cost for SlateQ:
1.44118
